In [1]:
import numpy as np

def simulate_experiment(sol1, sol2):
    # Hypothetical true relationship
    true_peak = 100 * np.exp(-((sol1-50)**2 + (sol2-50)**2)/400)
    # Adding random noise
    noise = np.random.normal(0, 5)
    return true_peak + noise


In [2]:
simulate_experiment(50, 50)

89.99266944673413

In [6]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# Kernel: composed of a constant term and an RBF term for smoothness
kernel = C(1.0, (1e-4, 1e1)) * RBF(10, (1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=5.0)


In [12]:
def expected_improvement(X, X_sample, Y_sample, gp, xi=0.01):
    '''
    Computes the EI at points X based on existing samples X_sample
    and Y_sample using a Gaussian process surrogate model gp.
    '''
    mu, sigma = gp.predict(X, return_std=True)
    mu_sample = gp.predict(X_sample)

    sigma = sigma.reshape(-1, 1)
    
    # Needed for noise-based model stability
    sigma = np.maximum(sigma, 1e-9)
    
    mu_sample_opt = np.max(mu_sample) # best sample value so far. Y_best

    
    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi # improvement = mean - best sample value so far - xi
        Z = imp / sigma # Z = improvement / standard deviation. convert improvement to Z-score
        # norm.cdf(Z) is probability that normal variable is less than Z.
        # norm.pdf(Z) is probability density function of normal variable at Z.
        # imp * norm.cdf(Z) expected gain from improvement, weighted by the probability that any improvement will actually occur.
        # sigma * norm.pdf(Z) qunatifies value of reducing uncertainty at X, regardless of whether the mean predicted improvement is positive.
        # a high uncertanty (sigma) at points where Z(and thus imp) is low can still result in a high expected improvement
        # because possibility of uncovering a significantly better or worse outcome than expected.
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
        #to finalize, imp*norm.cdf(Z) prioritizing areas with a high mean predicted improvement (exploiting known good areas).
        #sigma*norm.pdf(Z) prioritizing areas with high uncertainty (exploring unknown areas).
        #HIgher xi means more exploration, lower xi means more exploitation, lower xi means more exploitation.
    #improvement is defined as max(0, Y(x) - Y_best) where Y(x) is randomly observed value at X. 
#At any point X, calculate how much improvement is expected over the best observed outcome so far (Y_best). This is EI(X).
    return ei


In [13]:
from scipy.stats import norm
from scipy.optimize import minimize

def propose_location(acquisition, X_sample, Y_sample, gp):
    '''
    Proposes the next sampling point by optimizing the acquisition function.
    '''
    dim = X_sample.shape[1]
    min_val = 1
    min_x = None
    
    def min_obj(X):
        # convert max(0, Y(x) - Y_best) into a minimization problem
        return -acquisition(X.reshape(-1, dim), X_sample, Y_sample, gp)

    # Start from 10 different random points
    for x0 in np.random.uniform(0, 100, size=(10, dim)):
        #allowable range for the solution variables, which in this case are the volumes of solution 1 and solution 2 (both ranging from 0 to 100 mL).
        res = minimize(min_obj, x0=x0, bounds=[(0, 100), (0, 100)], method='L-BFGS-B')
        if res.fun < min_val:
            min_val = res.fun
            min_x = res.x
            
    return np.array([min_x])

# Initial random samples
X_sample = np.random.uniform(0, 100, (10, 2))
Y_sample = np.array([simulate_experiment(x[0], x[1]) for x in X_sample])

# Number of iterations
n_iter = 50

for i in range(n_iter):
    # Update Gaussian process with existing samples
    gp.fit(X_sample, Y_sample)

    # Using Expected Improvement to determine the next point to sample
    X_next = propose_location(expected_improvement, X_sample, Y_sample, gp)
    Y_next = simulate_experiment(X_next[0][0], X_next[0][1])

    # Add the new sample to existing
    X_sample = np.vstack((X_sample, X_next))
    Y_sample = np.append(Y_sample, Y_next)

    # Check if we are close to target
    if np.abs(Y_next - 100) < 1:  # tolerance could be adjusted
        print(f"Converged to within 1mA of the target after {i+1} iterations.")
        break

print("Final sample points:", X_sample)
print("Final measurements:", Y_sample)


/Users/kirillkalashnikov/Downloads/PSPythonSDK/myenv/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/kirillkalashnikov/Downloads/PSPythonSDK/myenv/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


For point  [[31.66615308  0.75461567]]  EI is  [[5.66460107e-91]]  with mu  [-1.1231015]  and sigma  [[2.40063989]]
For point  [[31.66615309  0.75461567]]  EI is  [[5.66460117e-91]]  with mu  [-1.1231015]  and sigma  [[2.40063989]]
For point  [[31.66615308  0.75461568]]  EI is  [[5.66460047e-91]]  with mu  [-1.1231015]  and sigma  [[2.40063989]]
For point  [[59.24387514 15.30273962]]  EI is  [[2.3835471e-48]]  with mu  [8.60961409]  and sigma  [[2.66415762]]
For point  [[59.24387515 15.30273962]]  EI is  [[2.38354702e-48]]  with mu  [8.60961409]  and sigma  [[2.66415762]]
For point  [[59.24387514 15.30273963]]  EI is  [[2.38354715e-48]]  with mu  [8.6096141]  and sigma  [[2.66415762]]
For point  [[65.98808989 84.09922585]]  EI is  [[1.28637546e-34]]  with mu  [13.55316404]  and sigma  [[2.77379545]]
For point  [[65.9880899  84.09922585]]  EI is  [[1.28637545e-34]]  with mu  [13.55316404]  and sigma  [[2.77379545]]
For point  [[65.98808989 84.09922586]]  EI is  [[1.28637543e-34]]  with 

/Users/kirillkalashnikov/Downloads/PSPythonSDK/myenv/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/kirillkalashnikov/Downloads/PSPythonSDK/myenv/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/kirillkalashnikov/Downloads/PSPythonSDK/myenv/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a be

For point  [[66.83066239 29.80806816]]  EI is  [[3.51155435e-213]]  with mu  [19.44049392]  and sigma  [[1.54762253]]
For point  [[66.8306624  29.80806816]]  EI is  [[3.51155291e-213]]  with mu  [19.4404939]  and sigma  [[1.54762253]]
For point  [[66.83066239 29.80806817]]  EI is  [[3.51155452e-213]]  with mu  [19.44049394]  and sigma  [[1.54762253]]
For point  [[38.51595087 85.57623691]]  EI is  [[6.4690508e-106]]  with mu  [6.8032125]  and sigma  [[2.79214699]]
For point  [[38.51595088 85.57623691]]  EI is  [[6.46905053e-106]]  with mu  [6.8032125]  and sigma  [[2.79214699]]
For point  [[38.51595087 85.57623692]]  EI is  [[6.46905085e-106]]  with mu  [6.80321249]  and sigma  [[2.79214699]]
For point  [[38.18590877 43.67867829]]  EI is  [[5.31907514e-37]]  with mu  [36.77130654]  and sigma  [[2.45222454]]
For point  [[38.18590878 43.67867829]]  EI is  [[5.31907575e-37]]  with mu  [36.77130657]  and sigma  [[2.45222454]]
For point  [[38.18590877 43.6786783 ]]  EI is  [[5.31907531e-37]]

/Users/kirillkalashnikov/Downloads/PSPythonSDK/myenv/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/kirillkalashnikov/Downloads/PSPythonSDK/myenv/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/kirillkalashnikov/Downloads/PSPythonSDK/myenv/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a be

...

/Users/kirillkalashnikov/Downloads/PSPythonSDK/myenv/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/kirillkalashnikov/Downloads/PSPythonSDK/myenv/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


For point  [[29.93824679 48.54037162]]  EI is  [[0.]]  with mu  [33.84444233]  and sigma  [[1.33796562]]
For point  [[10.64981735 55.53483033]]  EI is  [[0.]]  with mu  [3.05187071]  and sigma  [[2.28946953]]
For point  [[10.64981736 55.53483033]]  EI is  [[0.]]  with mu  [3.05187071]  and sigma  [[2.28946953]]
For point  [[10.64981735 55.53483034]]  EI is  [[0.]]  with mu  [3.05187071]  and sigma  [[2.28946953]]
For point  [[46.89761329 83.26116777]]  EI is  [[0.]]  with mu  [7.80416384]  and sigma  [[1.91424782]]
For point  [[46.8976133  83.26116777]]  EI is  [[0.]]  with mu  [7.80416384]  and sigma  [[1.91424782]]
For point  [[46.89761329 83.26116778]]  EI is  [[0.]]  with mu  [7.80416383]  and sigma  [[1.91424782]]
For point  [[39.87913233  2.14974499]]  EI is  [[4.33635357e-290]]  with mu  [1.02954689]  and sigma  [[2.56710005]]
For point  [[39.87913234  2.14974499]]  EI is  [[4.33635415e-290]]  with mu  [1.02954689]  and sigma  [[2.56710005]]
For point  [[39.87913233  2.149745  ]

/Users/kirillkalashnikov/Downloads/PSPythonSDK/myenv/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/kirillkalashnikov/Downloads/PSPythonSDK/myenv/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/kirillkalashnikov/Downloads/PSPythonSDK/myenv/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a be

For point  [[75.79453618 38.56267562]]  EI is  [[0.]]  with mu  [9.666875]  and sigma  [[1.91594618]]
For point  [[75.79453619 38.56267562]]  EI is  [[0.]]  with mu  [9.66687498]  and sigma  [[1.91594618]]
For point  [[75.79453618 38.56267563]]  EI is  [[0.]]  with mu  [9.66687501]  and sigma  [[1.91594618]]
For point  [[ 3.40714169 77.07341453]]  EI is  [[0.]]  with mu  [2.69343746]  and sigma  [[1.88808886]]
For point  [[ 3.4071417  77.07341453]]  EI is  [[0.]]  with mu  [2.69343746]  and sigma  [[1.88808886]]
For point  [[ 3.40714169 77.07341454]]  EI is  [[0.]]  with mu  [2.69343746]  and sigma  [[1.88808886]]
For point  [[29.76807255 97.4920422 ]]  EI is  [[0.]]  with mu  [3.23188334]  and sigma  [[1.88697999]]
For point  [[29.76807256 97.4920422 ]]  EI is  [[0.]]  with mu  [3.23188334]  and sigma  [[1.88697999]]
For point  [[29.76807255 97.49204221]]  EI is  [[0.]]  with mu  [3.23188334]  and sigma  [[1.88697999]]
For point  [[65.26084756 91.0483003 ]]  EI is  [[0.]]  with mu  [-

/Users/kirillkalashnikov/Downloads/PSPythonSDK/myenv/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [14]:
print("Final sample points:", X_sample)
print("Final measurements:", Y_sample)

Final sample points: [[ 7.96339864 17.43986647]
 [64.24813069 38.22993809]
 [57.24083038 36.32444532]
 [91.90063547 57.24001274]
 [58.10527953 60.93334911]
 [30.04548794 12.36481419]
 [73.87983389 27.85066521]
 [82.92843066 56.4803167 ]
 [50.58074593 73.55204906]
 [21.36967903 48.55681737]
 [59.76211788 73.56093729]
 [55.31900623 49.98568576]
 [42.35955121 38.2688434 ]
 [36.52344624 41.62450759]
 [44.91866338 31.85871172]
 [51.20108283 34.25764087]
 [51.88951974 48.86568115]
 [69.24988541 54.02588406]
 [17.61361    77.82295862]
 [51.93285462 59.63693516]
 [90.03346213 92.73429025]
 [34.51635516 99.75850507]
 [87.60884387  6.63196767]
 [61.20524956  5.03266754]
 [51.23193499 49.8640924 ]
 [42.53933676 42.07654505]
 [44.0477797  51.08561134]
 [41.77943246 75.31624221]
 [45.87936391 58.71905816]
 [60.47795599 46.47464537]
 [51.59036502 58.51274201]
 [ 9.78543992 95.51782524]
 [34.79403389 48.7159397 ]
 [42.09260192 53.03140317]
 [44.35741406 58.25537951]
 [54.45394187 54.57930379]
 [46.16